<a href="https://colab.research.google.com/github/koleshjr/COVID_19-TWEETS-CLASSIFICATION/blob/main/Covid19_tweets_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## COVID-19 Tweet Classification 

1.   List item

1.   List item
2.   List item


2.   List item


* Can you identify tweets about coronavirus without using keywords? 

In [1]:
# !pip install -q datasets
# !pip install transformers
# !pip install optuna
# !pip install sentencepiece
# !pip install evaluate


# !pip install mlflow wandb



In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pathlib import Path
path = '/content/drive/MyDrive/tweets_classification/'

In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import seaborn as sns 
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import mlflow
import os
import seaborn as sns
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import transformers 
import wandb

In [5]:
# !wandb login 
# wandb.init(project="classify_covid_tweets")

In [6]:
# %env WANDB_LOG_MODEL=true   #log every trained model

In [7]:

transformers.set_seed(1) 
df = pd.read_csv(path + 'Train.csv')
df

 





,ID,text,target
0,train_0,The bitcoin halving is cancelled due to,1
1,train_1,MercyOfAllah In good times wrapped in its gran...,0
2,train_2,266 Days No Digital India No Murder of e learn...,1
3,train_3,India is likely to run out of the remaining RN...,1
4,train_4,In these tough times the best way to grow is t...,0
...,...,...,...
5282,train_6856,The spread of the novel among asylum seekers o...,1
5283,train_6857,Hundreds of Jewish patients are being treated...,1
5284,train_6858,Beats me Honestly most of the people I follow ...,0
5285,train_6859,Help us reach more people by donating and shar...,0


In [8]:
eval_df = pd.read_csv(path + 'Test.csv')
len(eval_df)

1962

In [9]:
eval_df.head()

,ID,text
0,test_2,Why is explained in the video take a look
1,test_3,Ed Davey fasting for Ramadan No contest
2,test_4,Is Doja Cat good or do you just miss Nicki Minaj
3,test_8,How Boris Johnson s cheery wounded in action p...
4,test_9,Man it s terrible Not even a reason to get on ...


In [10]:
df.target.value_counts()


0    2746
1    2541
Name: target, dtype: int64

### Training

In [11]:
model_nm = "bert-base-uncased"

tokz = AutoTokenizer.from_pretrained(model_nm)



model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2, hidden_dropout_prob=0.2)#, gradient_checkpointing = True).to(device)
# gradient_checkpointing: for saving memory - but will make computation slower

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
# In the first step we will split the data in training and remaining dataset
df = df.rename(columns = {'target':'label'})
df_train, df_test = train_test_split(df, train_size=0.8, stratify = df['label'])



df_train.shape, df_test.shape, eval_df.shape


((4229, 3), (1058, 3), (1962, 2))

In [13]:


adverse = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "valid": Dataset.from_pandas(df_test),
    "test": Dataset.from_pandas(eval_df)
    })



Tokenizing function

In [14]:
adverse

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'label', '__index_level_0__'],
        num_rows: 4229
    })
    valid: Dataset({
        features: ['ID', 'text', 'label', '__index_level_0__'],
        num_rows: 1058
    })
    test: Dataset({
        features: ['ID', 'text'],
        num_rows: 1962
    })
})

In [15]:
def tokenize(x): return tokz(x["text"])



adverse_encoded = adverse.map(tokenize, batched=True, batch_size=8,remove_columns=('text','ID')) #, 


  0%|          | 0/529 [00:00<?, ?ba/s]

  0%|          | 0/133 [00:00<?, ?ba/s]

  0%|          | 0/246 [00:00<?, ?ba/s]

In [16]:
batch_size = 16
# the default logging_steps in TrainingArguments is set to 500 steps, so no loss is reported before 500 steps
model_name = f"{model_nm}-finetuned-adverse"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=3,
                                  learning_rate= 4e-5,
                                  per_device_train_batch_size=batch_size,
#                                   per_device_eval_batch_size=batch_size,
                                  weight_decay=0.03,
                                  evaluation_strategy="epoch",
                                logging_strategy="epoch",
                                  disable_tqdm=False,
#                                   auto_find_batch_size=True,
#                                   logging_steps=logging_steps,
#                                   push_to_hub=True,
                                  log_level="error",
                                 fp16 = True,  # for saving memory
                                 gradient_accumulation_steps=8, # for saving memory
                                  gradient_checkpointing=True, # for saving memory
                                 report_to="wandb")
#                                  optim="adafactor",) # saves memory bas a worse optimizer 

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    f1 = f1_score(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [18]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=adverse_encoded["train"],
                  eval_dataset=adverse_encoded["valid"],
                  tokenizer=tokz)
trainer.train();  



/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koleshjr (team-kolesh). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.413200,0.263501,0.902647,0.897103
1,0.220900,0.248351,0.904537,0.897462
2,0.155000,0.224461,0.918715,0.915354


In [19]:
trainer.evaluate()

{'eval_loss': 0.22446149587631226,
 'eval_accuracy': 0.9187145557655955,
 'eval_f1': 0.9153543307086615,
 'eval_runtime': 2.1131,
 'eval_samples_per_second': 500.692,
 'eval_steps_per_second': 62.941,
 'epoch': 3.0}

### Optuna

In [20]:


def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2, gradient_checkpointing = True)#.to(device)



In [21]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3 , log=True),
#         "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 5),
#         "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
    }


In [22]:


def my_objective(metrics):
    return metrics["eval_f1"]



In [24]:
trainer = Trainer(model_init=model_init,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=adverse_encoded["train"],
                  eval_dataset=adverse_encoded["valid"],
                  tokenizer=tokz)

In [25]:


best_run = trainer.hyperparameter_search(direction="maximize", compute_objective=my_objective, n_trials=5, hp_space = my_hp_space)



[I 2023-02-21 14:33:02,147] A new study created in memory with name: no-name-c6d3bf36-7783-4f85-af6f-d9999b88db83
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


eval/accuracy,▁▂██
eval/f1,▁▁██
eval/loss,█▅▁▁
eval/runtime,█▇▂▁
eval/samples_per_second,▁▂▇█
eval/steps_per_second,▁▂▇█
train/epoch,▁▁▅▅████
train/global_step,▁▁▅▅████
train/learning_rate,█▅▁
train/loss,█▃▁
train/total_flos,▁


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.801000,0.692739,0.519849,0.000000
1,0.727300,0.693495,0.519849,0.000000
2,0.698200,0.692206,0.519849,0.000000


[I 2023-02-21 14:34:15,661] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.0005910646892086325, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.0.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


eval/accuracy,▁▁▁
eval/f1,▁▁▁
eval/loss,▄█▁
eval/runtime,█▃▁
eval/samples_per_second,▁▆█
eval/steps_per_second,▁▆█
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▅▁
train/loss,█▃▁
train/total_flos,▁


...

[I 2023-02-21 14:35:50,629] Trial 1 finished with value: 0.7317073170731707 and parameters: {'learning_rate': 2.7714823900494865e-06, 'per_device_train_batch_size': 16}. Best is trial 1 with value: 0.7317073170731707.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


eval/accuracy,▁▇█
eval/f1,▁▇█
eval/loss,█▃▁
eval/runtime,▅▁█
eval/samples_per_second,▃█▁
eval/steps_per_second,▃█▁
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/learning_rate,█▅▁
train/loss,█▄▁
train/total_flos,▁


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.681500,0.633785,0.752363,0.698157


[W 2023-02-21 14:36:35,442] Trial 2 failed with parameters: {'learning_rate': 4.7782053110163305e-06, 'per_device_train_batch_size': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.8/dist-packages/transformers/integrations.py", line 183, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1543, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1791, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 2539, in training_step
    loss = self.compute_loss(model, inputs)
  File "/usr/local/lib/python3.8/dist-packa

KeyboardInterrupt: ignored

In [26]:


wandb.finish()



eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/learning_rate,▁
train/loss,▁
eval/accuracy,0.75236
